In [71]:
import pandas as pd
import numpy as np
import pybaseball

In [102]:
from pybaseball import pitching_stats
pitcher_data = pitching_stats(2022, qual=5)
pitcher_data = pitcher_data[['IDfg', 'Name', 'Age', 'G', 'IP', 'WAR']]

In [103]:
from pybaseball import batting_stats
batter_data = batting_stats(2022, qual=20)
batter_data = batter_data[['IDfg', 'Name', 'Age', 'G', 'AB', 'WAR']]

In [111]:
batter_data

,IDfg,Name,Age,G,AB,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023
0,15640,Aaron Judge,30,157,570,11.6,judge,aaron,judgeaa01,15640,2016.0,"$40,000,000"
1,11493,Manny Machado,29,150,578,7.5,machado,manny,machama01,11493,2012.0,"$21,090,909"
2,9777,Nolan Arenado,31,148,557,7.2,arenado,nolan,arenano01,9777,2013.0,"$32,822,071"
3,5361,Freddie Freeman,32,159,612,7.1,freeman,freddie,freemfr01,5361,2010.0,"$25,290,674"
4,9218,Paul Goldschmidt,34,151,561,7.0,goldschmidt,paul,goldspa01,9218,2011.0,"$25,333,333"
...,...,...,...,...,...,...,...,...,...,...,...,...
389,12775,Brad Miller,32,81,222,-1.2,miller,brad,millebr02,12775,2013.0,"$4,000,000"
390,19864,Riley Adams,26,48,142,-1.2,adams,riley,adamsri03,19864,2021.0,"$728,800"
391,23378,Jose Barrero,24,48,165,-1.2,barrero,josé,garcijo02,23378,2020.0,"$730,000"
392,11680,Elias Diaz,31,105,351,-1.5,díaz,elías,diazel01,11680,2015.0,"$5,500,000"


In [105]:
salaries = pd.read_csv("../Data/MLB-Salaries 2000-23 - 2023.csv", header= 1).drop("MLS", axis = 1)
salaries = salaries[salaries['2023'].notna()]

In [106]:
appearances = pd.read_csv("../Data/PositionAppearances2022.csv")
appearances = appearances.drop(['yearID','teamID', 'lgID', 'G_batting', 'G_defense', "G_ph", "G_pr"], axis = 1).groupby('playerID', as_index=False).sum() 

In [107]:
from pybaseball import playerid_reverse_lookup
lookuptable = playerid_reverse_lookup(appearances['playerID'], key_type='bbref')

batter_data = batter_data.merge(lookuptable, left_on= "IDfg", right_on= "key_fangraphs")
batter_data.drop(['key_retro', 'key_mlbam', 'mlb_played_last'], axis = 1, inplace= True)
pitcher_data = pitcher_data.merge(lookuptable, left_on= "IDfg", right_on= "key_fangraphs")
pitcher_data.drop(['key_retro', 'key_mlbam', 'mlb_played_last'], axis = 1, inplace= True)


In [65]:
# This takes a while
#from pybaseball import playerid_lookup
#df = pd.DataFrame()
#for id, row in salaries.iterrows():
#    last = f"{row['Last']}"
#    first = f"{row['First'].strip()}" 
#    a = playerid_lookup(last, first, fuzzy= True).iloc[[0]]
#    df = pd.concat([df, a], ignore_index=True)
    

In [108]:
#df.to_csv('../Data/SalaryLookup.csv')
df = pd.read_csv('../Data/SalaryLookup.csv')

In [109]:
salaries["key_fangraphs"] = df["key_fangraphs"]

In [110]:
batter_data = batter_data.merge(salaries[['2023', 'key_fangraphs']],  on= "key_fangraphs")
pitcher_data = pitcher_data.merge(salaries[['2023', 'key_fangraphs']],  on= "key_fangraphs")

In [112]:
import statsapi
teams = statsapi.lookup_team("", activeStatus="Y") 

In [113]:
team_ids = [team['id'] for team in teams]

In [114]:
# Pulling rosters from Opening Day 2023
rosters_df = pd.DataFrame()
for id in team_ids: 
    roster_string = statsapi.roster(id,'40Man',date=statsapi.get('season',{'seasonId':2023,'sportId':1})['seasons'][0]['regularSeasonStartDate'])
    rows = [line.split(maxsplit=2) for line in roster_string.strip().split('\n')]
    df = pd.DataFrame(rows, columns=['Number', 'Position', 'Player'])
    df[['First Name', 'Last Name']] = df['Player'].str.split(n=1, expand=True)
    df['teamid'] = id
    rosters_df = pd.concat([rosters_df, df], ignore_index=True)


In [115]:
rosters_df
rosters_df = rosters_df.merge(pd.DataFrame(teams)[['id', 'shortName']], left_on= "teamid", right_on= "id").drop(["id"], axis = 1)


In [118]:
# This takes a while - mapping players with teams back to ids 
#from pybaseball import playerid_lookup
#mapped_rosters = pd.DataFrame()
#for id, row in rosters_df.iterrows():
#    last = f"{row['Last Name']}"
#    first = f"{row['First Name'].strip()}" 
#    a = playerid_lookup(last, first, fuzzy= True).iloc[[0]] 
#    a['Last Name'] = row['Last Name']
#    a['First Name'] = row['First Name']
#    a['shortName'] = row['shortName']
#    a['Number'] = row['Number']            
#    mapped_rosters = pd.concat([mapped_rosters, a], ignore_index=True)

In [119]:
#mapped_rosters.to_csv('../Data/RosterLookup.csv')
mapped_rosters = pd.read_csv('../Data/RosterLookup.csv')

In [122]:
mapped_rosters = mapped_rosters[['key_fangraphs', 'shortName', ]]

In [123]:
mapped_rosters

,key_fangraphs,shortName
0,20132,Oakland
1,21023,Oakland
2,15937,Oakland
3,19627,Oakland
4,22411,Oakland
...,...,...
1234,15124,NY Mets
1235,13755,NY Mets
1236,1157,NY Mets
1237,2967,NY Mets


In [125]:
batter_data = batter_data.merge(mapped_rosters ,  on= "key_fangraphs")
pitcher_data = pitcher_data.merge(mapped_rosters , on= "key_fangraphs")

In [128]:
batter_data

,IDfg,Name,Age,G,AB,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023,shortName
0,11493,Manny Machado,29,150,578,7.5,machado,manny,machama01,11493,2012.0,"$21,090,909",San Diego
1,9777,Nolan Arenado,31,148,557,7.2,arenado,nolan,arenano01,9777,2013.0,"$32,822,071",St. Louis
2,5361,Freddie Freeman,32,159,612,7.1,freeman,freddie,freemfr01,5361,2010.0,"$25,290,674",LA Dodgers
3,9218,Paul Goldschmidt,34,151,561,7.0,goldschmidt,paul,goldspa01,9218,2011.0,"$25,333,333",St. Louis
4,19556,Yordan Alvarez,25,135,470,6.7,álvarez,yordan,alvaryo01,19556,2019.0,"$7,833,333",Houston
...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,12775,Brad Miller,32,81,222,-1.2,miller,brad,millebr02,12775,2013.0,"$4,000,000",Texas
385,19864,Riley Adams,26,48,142,-1.2,adams,riley,adamsri03,19864,2021.0,"$728,800",Washington
386,23378,Jose Barrero,24,48,165,-1.2,barrero,josé,garcijo02,23378,2020.0,"$730,000",Cincinnati
387,11680,Elias Diaz,31,105,351,-1.5,díaz,elías,diazel01,11680,2015.0,"$5,500,000",Colorado


In [130]:
pitcher_data

,IDfg,Name,Age,G,IP,WAR,name_last,name_first,key_bbref,key_fangraphs,mlb_played_first,2023,shortName
0,16149,Aaron Nola,29,32,205.0,6.3,nola,aaron,nolaaa01,16149,2015.0,"$16,250,000",Philadelphia
1,16137,Carlos Rodon,29,31,178.0,6.2,rodón,carlos,rodonca01,16137,2015.0,"$22,833,333",NY Yankees
2,8700,Justin Verlander,39,28,175.0,6.0,verlander,justin,verlaju01,8700,2005.0,"$43,333,333",NY Mets
3,18684,Sandy Alcantara,26,32,228.2,5.8,alcántara,sandy,alcansa01,18684,2017.0,"$6,300,000",Miami
4,14107,Kevin Gausman,31,31,174.2,5.7,gausman,kevin,gausmke01,14107,2013.0,"$21,000,000",Toronto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,18694,Kolby Allard,24,10,21.0,-0.7,allard,kolby,allarko01,18694,2018.0,"$750,000",Atlanta
439,17490,Genesis Cabrera,25,39,44.2,-0.8,cabrera,génesis,cabrege01,17490,2019.0,"$950,000",St. Louis
440,16933,Elieser Hernandez,27,20,62.1,-0.9,hernández,elieser,hernael01,16933,2018.0,"$1,600,000",NY Mets
441,6986,Ian Kennedy,37,57,50.1,-1.0,kennedy,ian,kenneia01,6986,2007.0,"$2,250,000",Texas
